In [4]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import tensorflow as tf

In [3]:
df = pd.read_csv("sign_data_ver1/train_data.csv")
df.head()

,068/09_068.png,068_forg/03_0113068.PNG,1
0,068/09_068.png,068_forg/01_0124068.PNG,1
1,068/09_068.png,068_forg/02_0124068.PNG,1
2,068/09_068.png,068_forg/01_0113068.PNG,1
3,068/09_068.png,068_forg/04_0124068.PNG,1
4,068/09_068.png,068_forg/04_0113068.PNG,1


In [6]:
path = 'sign_data_ver1/train/'


In [7]:
x = []
y = []
for ind in tqdm(df.index):
    name1 = df['068/09_068.png'][ind]
    name2 = df['068_forg/03_0113068.PNG'][ind]
    img1 = cv2.cvtColor(cv2.imread(path+str(name1)), cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path+str(name2)), cv2.COLOR_BGR2GRAY)
    img1 = np.array(img1).astype('float32')/255
    img2 = np.array(img2).astype('float32')/255
    img1 = cv2.resize(img1, (128,128), cv2.INTER_CUBIC)
    img2 = cv2.resize(img2, (128,128), cv2.INTER_CUBIC)
    x += [[img1, img2]]
    y += [df['1'][ind]]

 53%|█████▎    | 12308/23205 [01:39<01:27, 124.15it/s]


KeyboardInterrupt: 

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x_train1 = x[:17000,0]
x_train2 = x[:17000,1]

In [5]:
from keras import Input
from keras import layers
from keras import regularizers

In [6]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

In [7]:
input = layers.Input((128, 128, 1))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(32, (5, 5), activation="relu",kernel_regularizer=regularizers.L2(l2=2e-4),
    bias_regularizer=regularizers.L2(2e-4))(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(32, (5, 5),kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(2e-4), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)


x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(128, activation="relu")(x)
embedding_network = tf.keras.Model(input, x)


input_1 = layers.Input((128, 128, 1))
input_2 = layers.Input((128, 128, 1))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [9]:
siamese.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
siamese.load_weights("siamese_model1.h5")
siamese.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 128)          3578980     ['input_2[0][0]',                
                                                                  'input_3[0][0]']          

In [ ]:
siamese.fit(
    [x_train1, x_train2],
    y,
    validation_split = 0.1,
    batch_size = 32,
    epochs = 10,
    verbose = 1
)